In [ ]:
# Quantopian code
# Please copy and paste it on Quantopian IDE

import numpy as np

def initialize(context):
    schedule_function(check_pairs, date_rules.every_day(), time_rules.market_close(minutes=60))
        
    # Canadian Natural Resources
    context.cnq = sid(21735)
    
    # Pioneer Natural Resources Company
    context.pxd = sid(17436)
    
    context.long_on_spread = False
    context.shorting_spread = False
    
    
def check_pairs(context, data):
    cnq = context.cnq
    pxd = context.pxd
    
    prices = data.history([cnq, pxd], 'price', 30, '1d')
    
    short_prices = prices.iloc[-1:]
    
    # Spread
    mavg_30 = np.mean(prices[cnq] - prices[pxd])
    std_30 = np.std(prices[cnq] - prices[pxd])
    mavg_1 = np.mean(short_prices[cnq] - short_prices[pxd])
    
    if std_30 > 0:
        zscore = (mavg_1 - mavg_30) / std_30
        
        # Spread = CNQ - PXD
        if zscore > 1.2 and not context.shorting_spread:
            order_target_percent(cnq, -0.5) 
            order_target_percent(pxd, 0.5) 
            context.shorting_spread = True
            context.long_on_spread = False
            
        elif zscore < 1.2 and not context.long_on_spread:
            order_target_percent(cnq, 0.5) 
            order_target_percent(pxd, -0.5) 
            context.shorting_spread = False
            context.long_on_spread = True
            
        elif abs(zscore) < 0.1:
            order_target_percent(cnq, 0)
            order_target_percent(pxd, 0)
            context.shorting_spread = False
            context.long_on_spread = False
                
        record(Z_score = zscore)